# RADP RF Digital Twin + CCO Demo


# Prerequisite

### Ensure to install ffmpeg on your machine and create .env

Provide the executable ffmpeg file path in .env file.

Create .env file in root folder of repo and add below to env

`FFMPEG_PATH="/path_to_ffmpeg/ffmpeg"`


### Sample data set

Unpack the sample data set present at `notebooks/data/sim_data.zip` under `notebooks/data/`


In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

In [ ]:
import numpy as np
from scipy import stats
from scipy.ndimage import correlate
from skimage.metrics import structural_similarity as ssim
from radp_library import *

## Using pregenerated data stored locally
Currently the data is stored under notebooks

/data folder

In [ ]:
WORKING_DIR = f"{Path().absolute()}"
BUCKET_PATH = f"{WORKING_DIR}/data"
SIM_DATA_PATH = "sim_data/3cell"

## Bayesian digital twin training

In [ ]:
# provide list of folder name under which the pregenerated data is stored
sim_idx_folders = ['sim_001', 'sim_002', 'sim_003', 'sim_004', 'sim_005']

In [ ]:
p_train_maxiter_dict = {
        10: [1, 2],
        40: [10]
}
p_test = 100

pred_rsrp_list = []
MAE_list = []
Percentile85Error_list = []
p_train_list = []
maxiter_list = []


for p_train in p_train_maxiter_dict.keys():
    for maxiter in p_train_maxiter_dict[p_train]:
        logging.info(f"\n\nMAXITER = {maxiter}, p_train={p_train}\n")
        _, site_config_df, test_data, loss_vs_iter, lons, lats, true_rsrp, pred_rsrp, MAE, Percentile85Error = bdt(
            bucket_path=BUCKET_PATH,
            sim_data_path=SIM_DATA_PATH,
            p_train=p_train,
            p_test=p_test,
            maxiter=maxiter,
            sim_idx_folders=sim_idx_folders,
            test_idx=2,
            plot_loss_vs_iter=True,
            choose_strongest_samples_percell=False,
            filter_out_samples_dbm_threshold=-70,
            filter_out_samples_kms_threshold=0.65,
            # track_sampling=True,
            # num_UEs=2,
            # ticks=100,
        )
        p_train_list.append(p_train)
        maxiter_list.append(maxiter)
        pred_rsrp_list.append(pred_rsrp)
        MAE_list.append(MAE)
        Percentile85Error_list.append(Percentile85Error)

df_results = pd.DataFrame(
    {
        "pred_rsrp_list": pred_rsrp_list,
        "MAE_list": MAE_list,
        "Percentile85Error_list": Percentile85Error_list,
        "maxiter_list": maxiter_list,
        "p_train_list": p_train_list,
    }
)

In [ ]:
def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
def get_2draster(lats ,lons, vals, z):
    min_lon_tile, min_lat_tile = lon_lat_to_bing_tile(min(lons), min(lats), level=z)
    max_lon_tile, max_lat_tile = lon_lat_to_bing_tile(max(lons), max(lats), level=z)

    val_dict = {}
    for lon, lat, val in zip(lons, lats, vals):
        lon_idx, lat_idx = lon_lat_to_bing_tile(lon, lat, level=z)
        row_idx = lat_idx - max_lat_tile
        col_idx = lon_idx - min_lon_tile
        if row_idx not in val_dict:
            val_dict[row_idx] = {}
        if col_idx not in val_dict[row_idx]:
            val_dict[row_idx][col_idx] = []
        val_dict[row_idx][col_idx].append(val)

    num_cols = max_lon_tile - min_lon_tile + 1
    num_rows = min_lat_tile - max_lat_tile + 1
    raster = np.empty((num_rows, num_cols))
    raster.fill(min(true_rsrp))
    for lat_idx in val_dict.keys():
        for lon_idx in val_dict[lat_idx].keys():
            if val_dict[lat_idx][lon_idx] == []:
                raster[lat_idx][lon_idx] = min(true_rsrp)
            else:
                raster[lat_idx][lon_idx] = np.average(val_dict[lat_idx][lon_idx])

    return raster

In [ ]:
df_results = pd.DataFrame(
    {
        "pred_rsrp_list": pred_rsrp_list,
        "MAE_list": MAE_list,
        "Percentile85Error_list": Percentile85Error_list,
        "maxiter_list": maxiter_list,
        "p_train_list": p_train_list,
    }
)

df_results["pearson_corr"] = df_results.apply(
    lambda x: stats.pearsonr(true_rsrp, x.pred_rsrp_list)[0], axis=1
)

perc = 99
df_results["tail_error"] = df_results.apply(
    lambda x: np.percentile(abs(true_rsrp - x.pred_rsrp_list), perc), axis=1
)


lambda_corr = 0.8

df_results["MAE_list_normalized_inverted"] = 1 - normalize_data(df_results[['MAE_list']])

df_results["tail_error_normalized_inverted"] = 1 - normalize_data(df_results[['tail_error']])

df_results["pearson_corr_normalized"] = normalize_data(df_results[['pearson_corr']])

df_results["Percentile85Error_list_normalized_inverted"] = 1 - normalize_data(df_results[['Percentile85Error_list']])

df_results["pearson_MAE_inverted_normalized_mean"] = df_results[['pearson_corr_normalized', 'MAE_list_normalized_inverted']].mean(axis=1)

df_results["pearson_Percentile85Error_inverted_normalized_mean"] = df_results[['pearson_corr_normalized','Percentile85Error_list_normalized_inverted']].mul(
    (lambda_corr, 1 -lambda_corr)
).sum(1)

df_results["pearson_tail_error_inverted_normalized_mean"] = df_results[['pearson_corr_normalized', 'tail_error_normalized_inverted']].mean(axis=1)


df_results["variance_error"] = df_results.apply(
    lambda x: np.var(true_rsrp - x.pred_rsrp_list), axis=1
)

df_results_sorted = df_results


anim, fig = animate_predictions(
    lats,
    lons,
    true_rsrp,
    df_results_sorted.pred_rsrp_list.to_list(),
    df_results_sorted.MAE_list.to_list(),
    df_results_sorted.Percentile85Error_list.to_list(),
    df_results_sorted.maxiter_list.to_list(),
    df_results_sorted.p_train_list.to_list(),
    "/tmp/animation_digital_twin_larger_network_iter_scanning_v4.mp4",
)

In [ ]:
raster_true = get_2draster(lats, lons, df_results.pred_rsrp_list[2], 19)
weights = [[0, 1, 0],
           [1, 1, 1],
           [0, 1, 0]]
raster_true_smoothed = correlate(raster_true, weights, mode='nearest')
plt.imshow(raster_true_smoothed)

plt.plot(df_results["ssim"])

In [ ]:
plt.hist(abs(true_rsrp - df_results_sorted.pred_rsrp_list.iloc[2]))
print("Error = ", df_results_sorted.MAE_list.iloc[2])
print("Correlation = ", df_results_sorted.pearson_corr.iloc[2])
print("25th percentile = ", np.percentile(abs(true_rsrp - df_results_sorted.pred_rsrp_list.iloc[2]), 25))

In [ ]:
plt.hist(abs(true_rsrp - df_results_sorted.pred_rsrp_list.iloc[2]))
print("Error = ", df_results_sorted.MAE_list.iloc[2])
print("Correlation = ", df_results_sorted.pearson_corr.iloc[2])
print("25th percentile = ", np.percentile(abs(true_rsrp - df_results_sorted.pred_rsrp_list.iloc[2]), 25))

In [ ]:
df_results_sorted.MAE_list

In [ ]:
maxiters=[38]
p_train_list=[40]
_, site_config_df, test_data, loss_vs_iter, lons, lats, true_rsrp, pred_rsrp, MAE, Percentile85Error = bdt(
    bucket_path=BUCKET_PATH,
    sim_data_path=SIM_DATA_PATH,
    p_train=p_train_list[0],
    p_test=100,
    maxiter=maxiters[0],
    sim_idx_folders=sim_idx_folders,
    test_idx=2,
    plot_loss_vs_iter=True,
    choose_strongest_samples_percell=False,
    filter_out_samples_dbm_threshold=-70,
    filter_out_samples_kms_threshold=0.65,
)
animate_predictions(
    lats,
    lons,
    true_rsrp,
    [pred_rsrp],
    [MAE],
    [Percentile85Error],
    maxiters,
    p_train_list,
    "/tmp/animation_digital_twin_larger_network_iter_scanning_v5.mp4",
)

In [ ]:
df_results_sorted.pearson_corr

In [ ]:
df1 = pd.DataFrame(
    {
        "loc_x": [1.1, 2.3, 4.5],
        "loc_y": [-7.005, 68, 2.22],
        "pred_rsrp": [10, 10, 10],
        "cell_rxpwr_dbm": [12.0, 12.0, 12.0],
    }
)

df2 = pd.DataFrame(
    {
        "loc_x": [4.5, 1.1, 70.004],
        "loc_y": [2.22, -7.005, 2.22],
        "pred_rsrp": [20, 20, 20],
        "cell_rxpwr_dbm": [12.0000, 12.00001, 12.00001],
    }
)

pd.concat([df1, df2]).groupby(["loc_x", "loc_y"], as_index=False)[["pred_rsrp", "cell_rxpwr_dbm"]].max()

print(df1.apply(lon_lat_to_bing_tile_df_row, level=20, axis=1))
print(df1.apply(lon_lat_to_bing_tile_df_row, level=20, axis=1).apply(bing_tile_to_center_df_row, level=20, axis=1))
print(
    df1.apply(lon_lat_to_bing_tile_df_row, level=20, axis=1)
    .apply(bing_tile_to_center_df_row, level=20, axis=1)
    .apply(lon_lat_to_bing_tile_df_row, level=20, axis=1)
)
print(
    df1.apply(lon_lat_to_bing_tile_df_row, level=20, axis=1)
    .apply(bing_tile_to_center_df_row, level=20, axis=1)
    .apply(lon_lat_to_bing_tile_df_row, level=20, axis=1)
    .apply(bing_tile_to_center_df_row, level=20, axis=1)
)

In [ ]:
lons = test_data[1]["loc_x"].values
lats = test_data[1]["loc_y"].values

step = SRTM_STEP * 1.1

min_lon = min(lons)
max_lon = max(lons)
min_lat = min(lats)
max_lat = max(lats)
lon_dims = math.ceil((max_lon - min_lon) / step)
lat_dims = math.ceil((max_lat - min_lat) / step)

rf_raster = np.empty([lat_dims, lon_dims], dtype="float32")

for i in range(len(lons)):
    lon_idx = int((lons[i] - min_lon) / step)
    lat_idx = int((lats[i] - min_lat) / step)
    rf_raster[lat_idx][lon_idx] = pred_rsrp[i]

In [ ]:
shapes = rfco_to_best_server_shapes(
    spwr_src_raster=rf_raster,
    min_lat=min_lat,
    min_lon=min_lon,
    step=step,
)
raw_polys = [geometry.shape(shape[0]) for shape in shapes]

In [ ]:
len(raw_polys)

In [ ]:
# colors in ABGR order
line_style = fastkml.styles.LineStyle(color="7FFF3355", width=1)
poly_style = fastkml.styles.PolyStyle(color="88FFEEEE", fill=1, outline=1)
styles = fastkml.styles.Style(KML_NS, styles=[line_style, poly_style])

ShapesKMLWriter.shape_dict_to_kmz(
    {str(k): poly for k, poly in enumerate(raw_polys)},
    "shapes.kmz",
    styles=[styles],
    zipped=False,  # write as KML instead of KMZ for Mapbox viz
)

In [ ]:
plt.imshow(rf_raster)